### Simple object count for DP1
Based on a notebook from Perter Ferguson and Douglas Tucker.

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from astropy.table import vstack
import warnings
warnings.filterwarnings('ignore')

from lsst.daf.butler import Butler

In [ ]:
# these  are specific to NERSC
repo="/repo/DP1"
collection="LSSTComCam/DP1"
instrument = 'LSSTComCam'
skymap = 'lsst_cells_v1'




In [ ]:
tract_list = [453, 454, 4849, 5063, 4848, 2394, 2234, 4016, 4017, 4218, 4217, 5525, 5526, 7611, 7610, 7850, 10463, 10464, 10704]
tract_dict={453: '47 Tuc', 
            454: '47 Tuc',
           4849: 'ECDFS', 
           5063: 'ECDFS',
           4848: 'ECDFS', 
           2394: 'EDFS', 
           2234: 'EDFS',
           4016: 'Fornax', 
           4017: 'Fornax', 
           4218: 'Fornax', 
           4217: 'Fornax', 
           5525: 'Rubin_SV_095-25', 
           5526: 'Rubin_SV_095-25', 
           7611: 'Seagull', 
           7610: 'Seagull', 
           7850: 'Seagull',
           10463: 'Rubin_SV_38_7', 
           10464: 'Rubin_SV_38_7', 
           10704: 'Rubin_SV_38_7'
           }

In [ ]:
zeropoint = 31.4 # AB zeropoint, THIS IS POSSIBLY WRONG
def flux2mag(flux):
    return -2.5*np.log10(flux) + zeropoint
def fluxerr2magerr(flux,fluxerr):
    return ((-2.5*fluxerr)/(np.log(10)*flux))

In [ ]:
# specific to NERSC
os.environ['DAF_BUTLER_REPOSITORY_INDEX'] = '/global/cfs/cdirs/lsst/production/gen3/shared/data-repos.yaml'

In [ ]:
butler = Butler(repo, collections=collection)
assert butler is not None

In [ ]:
INCOLS = [
    'coord_ra',
    'coord_dec',
]
bands="ugrizy"
for band in bands:
    INCOLS += [
        f'{band}_psfFlux',
        f'{band}_psfFluxErr',
        #f'{band}_ap12Flux',
        #f'{band}_ap12FluxErr',
        f'{band}_gaap1p0Flux',
        f'{band}_gaap1p0FluxErr',
        f'{band}_extendedness',
        f'{band}_psfFlux_flag'
    ]


In [ ]:
comcam_galaxies_list = []
comcam_list = []
comcam_ecdfs_list = []
comcam_edfs_list = []

for tractId in tract_list:

    print()
    print(f'tract: {tractId}, field: {tract_dict[tractId]}')

    try:
    
        raw_comcam = butler.get('object', dataId={'skymap': skymap, 'tract': tractId}, 
                                collections=[collection],
                                parameters={"columns":INCOLS})

        # Insert tractId as the first column
        raw_comcam['tractId'] = tractId  
    
        # Insert field name -- if known -- as the second column
        if tractId in tract_dict:
            field = tract_dict[tractId]
        else:
            field = 'unknown'
        raw_comcam['field'] = field  

        # Clean the catalog
        #sel = (raw_comcam['detect_isPrimary'] == True)
        sel = (raw_comcam['r_psfFlux']/raw_comcam['r_psfFluxErr'] > 5)
        for band in ['g','r','i']:
            sel &= (raw_comcam[f'{band}_psfFlux_flag'] == 0)

        for band in bands:
            raw_comcam[f'{band}_gaap1p0Mag'] = flux2mag(raw_comcam[f'{band}_gaap1p0Flux'])
            raw_comcam[f'{band}_gaap1p0Magerr'] = fluxerr2magerr(raw_comcam[f'{band}_gaap1p0Flux'],raw_comcam[f'{band}_gaap1p0FluxErr'])

        comcam = raw_comcam[sel]

        # Find just the (most likely) stars...
        sel_comcam_galaxies =  (comcam['r_extendedness'] > 0.5) | (comcam['i_extendedness'] > 0.5)
        comcam_galaxies = comcam[sel_comcam_galaxies] 
        print(f"Number of objects: {len(comcam)}")
        print(f"Number of galaxies: {len(comcam_galaxies)}")

        # Append the dataframe to the list
        comcam_galaxies_list.append(comcam_galaxies) 
        comcam_list.append(comcam)
        if tract_dict[tractId] == 'ECDFS':
            comcam_ecdfs_list.append(comcam)
        elif tract_dict[tractId] == 'EDFS':
            comcam_edfs_list.append(comcam)

    # Catch any exception
    except Exception as e:

        print(f"An error occurred for tractId {tractId}: {e}")


# Concatenate all dataframes in the list
comcam_all = vstack(comcam_list)  
print(f"Total number of objects: {len(comcam_all)}")
comcam_galaxies_all = vstack(comcam_galaxies_list)  
print(f"Total number of galaxies: {len(comcam_galaxies_all)}")
comcam_ecdfs_all = vstack(comcam_ecdfs_list)
print(f"Total number of objects in ECDFS: {len(comcam_ecdfs_all)}")
comcam_edfs_all = vstack(comcam_edfs_list)
print(f"Total number of objects in EDFS: {len(comcam_edfs_all)}")
